In [ ]:
import os
import numpy as np
import librosa
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model
from tensorflow.keras import layers, models
from tensorflow.keras.applications import ResNet50
# virtual env: conda activate "K:\Code\Project\Research Paper\Emotion Detection\Code\resnet_env"

# -----------------------------
# Feature extraction
# -----------------------------
def extract_features(file_path, max_pad_len=174):
    signal, sr = librosa.load(file_path, sr=22050)
    mfcc = librosa.feature.mfcc(y=signal, sr=sr, n_mfcc=40)
    
    # Pad/truncate MFCC to fixed length for CNN
    if mfcc.shape[1] < max_pad_len:
        pad_width = max_pad_len - mfcc.shape[1]
        mfcc = np.pad(mfcc, pad_width=((0, 0), (0, pad_width)), mode='constant')
    else:
        mfcc = mfcc[:, :max_pad_len]
    return mfcc

# -----------------------------
# Dataset preparation
# -----------------------------
data_dirs = [
    "K:\\Code\\Project\\Research Paper\\Emotion Detection\\voice_data\\train_data\\SER",
    "K:\\Code\\Project\\Research Paper\\Emotion Detection\\archive\\audio_speech_actors_01-24\\Actor_01",
    "K:\\Code\\Project\\Research Paper\\Emotion Detection\\archive\\audio_speech_actors_01-24\\Actor_02",
    "K:\\Code\\Project\\Research Paper\\Emotion Detection\\archive\\audio_speech_actors_01-24\\Actor_03",
    "K:\\Code\\Project\\Research Paper\\Emotion Detection\\archive\\audio_speech_actors_01-24\\Actor_04",
    "K:\\Code\\Project\\Research Paper\\Emotion Detection\\archive\\audio_speech_actors_01-24\\Actor_05",
    "K:\\Code\\Project\\Research Paper\\Emotion Detection\\archive\\audio_speech_actors_01-24\\Actor_06",
    "K:\\Code\\Project\\Research Paper\\Emotion Detection\\archive\\audio_speech_actors_01-24\\Actor_07",
    "K:\\Code\\Project\\Research Paper\\Emotion Detection\\archive\\audio_speech_actors_01-24\\Actor_08",
    "K:\\Code\\Project\\Research Paper\\Emotion Detection\\archive\\audio_speech_actors_01-24\\Actor_09",
    "K:\\Code\\Project\\Research Paper\\Emotion Detection\\archive\\audio_speech_actors_01-24\\Actor_10",
    "K:\\Code\\Project\\Research Paper\\Emotion Detection\\archive\\audio_speech_actors_01-24\\Actor_11",
    "K:\\Code\\Project\\Research Paper\\Emotion Detection\\archive\\audio_speech_actors_01-24\\Actor_12",
    "K:\\Code\\Project\\Research Paper\\Emotion Detection\\archive\\audio_speech_actors_01-24\\Actor_13",
    "K:\\Code\\Project\\Research Paper\\Emotion Detection\\archive\\audio_speech_actors_01-24\\Actor_14",
    "K:\\Code\\Project\\Research Paper\\Emotion Detection\\archive\\audio_speech_actors_01-24\\Actor_15",
    "K:\\Code\\Project\\Research Paper\\Emotion Detection\\archive\\audio_speech_actors_01-24\\Actor_16",
    "K:\\Code\\Project\\Research Paper\\Emotion Detection\\archive\\audio_speech_actors_01-24\\Actor_17",
    "K:\\Code\\Project\\Research Paper\\Emotion Detection\\archive\\audio_speech_actors_01-24\\Actor_18",
    "K:\\Code\\Project\\Research Paper\\Emotion Detection\\archive\\audio_speech_actors_01-24\\Actor_19",
    "K:\\Code\\Project\\Research Paper\\Emotion Detection\\archive\\audio_speech_actors_01-24\\Actor_20",
    "K:\\Code\\Project\\Research Paper\\Emotion Detection\\archive\\audio_speech_actors_01-24\\Actor_21",
    "K:\\Code\\Project\\Research Paper\\Emotion Detection\\archive\\audio_speech_actors_01-24\\Actor_22",
    "K:\\Code\\Project\\Research Paper\\Emotion Detection\\archive\\audio_speech_actors_01-24\\Actor_23",
    "K:\\Code\\Project\\Research Paper\\Emotion Detection\\archive\\audio_speech_actors_01-24\\Actor_24",
    "K:\\Code\\Project\\Research Paper\\Emotion Detection\\archive2\\Angry",
    "K:\\Code\\Project\\Research Paper\\Emotion Detection\\archive2\\Happy",
    "K:\\Code\\Project\\Research Paper\\Emotion Detection\\archive2\\Natural",
    "K:\\Code\\Project\\Research Paper\\Emotion Detection\\archive2\\Sad",
    "K:\\Code\\Project\\Research Paper\\Emotion Detection\\archive2\\Surprised"

]
X, y, emotions = [], [], []

for data_dir in data_dirs:
    if not os.path.exists(data_dir):
        print(f"⚠️ Warning: {data_dir} does not exist, skipping...")
        continue

    for file in os.listdir(data_dir):
        if file.endswith(".wav"):
            label = file.split("_")[-1].replace(".wav", "")
            if label not in emotions:
                emotions.append(label)

            audio_path = os.path.join(data_dir, file)
            mfcc = extract_features(audio_path)
            X.append(mfcc)
            y.append(emotions.index(label))

X = np.array(X)
X = X[..., np.newaxis]  # CNN needs channel dimension
y = to_categorical(np.array(y), num_classes=len(emotions))

# -----------------------------
# Train/test split
# -----------------------------
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.15, random_state=None
)

# -----------------------------
# Build CNN model
# -----------------------------
def build_resnet_lstm_model(num_emotions):
    # Input shape = (40, 174, 1) → 40 MFCCs over 174 time frames
    input_layer = layers.Input(shape=(40, 174, 1))

    # Convert grayscale (1 channel) → 3 channels for ResNet
    x = layers.Conv2D(3, (1,1), padding='same')(input_layer)
    x = layers.Lambda(lambda img: tf.image.resize(img, (224, 224)))(x)


    # Pre-trained ResNet feature extractor (no top layers)
    base_model = ResNet50(include_top=False, weights='imagenet', input_shape=(224, 224, 3))
    


    # Optionally freeze first few layers (helps transfer learning)
    for layer in base_model.layers[:80]:
        layer.trainable = False

    # Extract features from ResNet
    x = base_model(x)              # shape: (None, H, W, C)
    x = layers.GlobalAveragePooling2D()(x)  # Flatten spatial dimensions
    x = layers.RepeatVector(40)(x)          # Repeat features for LSTM time steps

    # LSTM temporal modeling
    x = layers.LSTM(128, return_sequences=False)(x)
    x = layers.Dropout(0.3)(x)

    # Classification layers
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dropout(0.3)(x)
    output = layers.Dense(num_emotions, activation='softmax')(x)

    model = models.Model(inputs=input_layer, outputs=output)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model
# -----------------------------
# Train or load choice
# -----------------------------
choice = input("Do you want to load the saved model? (yes/no): ").strip().lower()
model_path = "K:\\Code\\Project\\Research Paper\\Emotion Detection\\Code\\resnet_tensor_emotion_model.keras"

if choice == "yes" and os.path.exists(model_path):
    print(" Loading saved model...")
    model = load_model(model_path)
    # also load emotions list
    import numpy as np
    emotions = np.load("emotions11.npy", allow_pickle=True).tolist()
else:
    print(" Training model from scratch...")
    model = build_resnet_lstm_model(len(emotions))
    model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100, batch_size=32)
    model.save(model_path)
    np.save("emotions11.npy", emotions)  # save label mapping

# -----------------------------
# Prediction function
# -----------------------------
def predict_emotion(file_path):
    mfcc = extract_features(file_path)
    mfcc = mfcc[np.newaxis, ..., np.newaxis]  # reshape for CNN
    prediction = model.predict(mfcc)
    return emotions[np.argmax(prediction)]

# -----------------------------
# Test prediction on new audio
# -----------------------------
test_file = "K:\\Code\\Project\\Research Paper\\Emotion Detection\\voice_data\\test_data\\self.wav"
print("Predicted Emotion:", predict_emotion(test_file))

In [3]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
from tqdm import tqdm
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import shutil

# ======================================================
# CONFIGURATION
# ======================================================
data_dirs = [
    r"K:\Code\Project\Research Paper\Emotion Detection\facial_data\archive\train",
    r"K:\Code\Project\Research Paper\Emotion Detection\facial_data\archive\test"
]
model_path = r"K:\Code\Project\Research Paper\Emotion Detection\Code\resnet_image_emotion_model.pth"
emotions_path = r"K:\Code\Project\Research Paper\Emotion Detection\Code\emotions_image.npy"
batch_size = 64
num_epochs = 30
learning_rate = 1e-4
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

torch.backends.cudnn.benchmark = True  # Optimize for GPU speed

# ======================================================
# MERGE DATASET
# ======================================================
all_images_dir = r"K:\Code\Project\Research Paper\Emotion Detection\facial_data\merged_dataset"
if not os.path.exists(all_images_dir):
    os.makedirs(all_images_dir, exist_ok=True)
    print("🔄 Combining all datasets into:", all_images_dir)

    for data_dir in data_dirs:
        if not os.path.exists(data_dir):
            print(f"⚠️ Warning: {data_dir} not found, skipping...")
            continue
        for emotion in os.listdir(data_dir):
            src_path = os.path.join(data_dir, emotion)
            if not os.path.isdir(src_path):
                continue
            dest_path = os.path.join(all_images_dir, emotion)
            os.makedirs(dest_path, exist_ok=True)
            for img_file in os.listdir(src_path):
                if img_file.lower().endswith(".jpg"):
                    shutil.copy2(
                        os.path.join(src_path, img_file),
                        os.path.join(dest_path, f"{emotion}_{img_file}")
                    )

# ======================================================
# TRANSFORMS (Data Augmentation)
# ======================================================
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

# ======================================================
# DATASETS & LOADERS
# ======================================================
full_dataset = datasets.ImageFolder(root=all_images_dir, transform=train_transform)
num_classes = len(full_dataset.classes)
print(f"✅ Loaded {len(full_dataset)} images | {num_classes} emotion classes: {full_dataset.classes}")

np.save(emotions_path, full_dataset.classes)

train_size = int(0.85 * len(full_dataset))
test_size = len(full_dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(full_dataset, [train_size, test_size])

# Update test dataset transform to non-augmented
test_dataset.dataset.transform = val_transform

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, pin_memory=True, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, pin_memory=True, num_workers=4)

# ======================================================
# MODEL SETUP
# ======================================================
def build_resnet(num_classes):
    model = models.resnet50(pretrained=True)

    # Unfreeze the last residual block for fine-tuning
    for name, param in model.named_parameters():
        param.requires_grad = ("layer4" in name) or ("fc" in name)

    in_features = model.fc.in_features
    model.fc = nn.Sequential(
        nn.Linear(in_features, 512),
        nn.BatchNorm1d(512),  # ✅ Batch normalization
        nn.ReLU(inplace=True),
        nn.Dropout(0.4),
        nn.Linear(512, num_classes)
    )
    return model


# ======================================================
# TRAINING OR LOADING
# ======================================================
choice = input("Do you want to load the saved model? (yes/no): ").strip().lower()

model = build_resnet(num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-5)
scaler = torch.cuda.amp.GradScaler()  # Mixed precision training
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)

best_val_acc = 0.0
train_losses, val_losses, train_accs, val_accs = [], [], [], []

if choice == "yes" and os.path.exists(model_path):
    print("📂 Loading saved model...")
    model.load_state_dict(torch.load(model_path, map_location=device))
else:
    print("🧠 Training model from scratch...")

    for epoch in range(num_epochs):
        model.train()
        running_loss, correct, total = 0.0, 0, 0

        train_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Training]", unit="batch")
        for images, labels in train_bar:
            images, labels = images.to(device, non_blocking=True), labels.to(device, non_blocking=True)
            optimizer.zero_grad(set_to_none=True)

            with torch.cuda.amp.autocast():
                outputs = model(images)
                loss = criterion(outputs, labels)

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            _, preds = torch.max(outputs, 1)
            running_loss += loss.item()
            correct += (preds == labels).sum().item()
            total += labels.size(0)
            train_bar.set_postfix(loss=f"{running_loss / (total // batch_size + 1):.4f}",
                                  acc=f"{100 * correct / total:.2f}%")

        train_loss = running_loss / len(train_loader)
        train_acc = 100 * correct / total

        # -------------------------
        # VALIDATION LOOP
        # -------------------------
        model.eval()
        val_loss, val_correct, val_total = 0.0, 0, 0
        with torch.no_grad():
            val_bar = tqdm(test_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Validation]", unit="batch")
            for images, labels in val_bar:
                images, labels = images.to(device, non_blocking=True), labels.to(device, non_blocking=True)
                with torch.cuda.amp.autocast():
                    outputs = model(images)
                    loss = criterion(outputs, labels)
                val_loss += loss.item()
                _, preds = torch.max(outputs, 1)
                val_correct += (preds == labels).sum().item()
                val_total += labels.size(0)
                val_bar.set_postfix(loss=f"{val_loss / (val_total // batch_size + 1):.4f}",
                                    acc=f"{100 * val_correct / val_total:.2f}%")

        val_loss /= len(test_loader)
        val_acc = 100 * val_correct / val_total

        train_losses.append(train_loss)
        val_losses.append(val_loss)
        train_accs.append(train_acc)
        val_accs.append(val_acc)

        print(f"\n📊 Epoch {epoch+1}/{num_epochs} Summary:")
        print(f"   🏋️ Train -> Loss: {train_loss:.4f}, Acc: {train_acc:.2f}%")
        print(f"   🔍 Val   -> Loss: {val_loss:.4f}, Acc: {val_acc:.2f}%\n")

        # Save best model
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), model_path)
            print(f"💾 Best model saved (Val Acc: {val_acc:.2f}%)")

        scheduler.step()

# ======================================================
# EVALUATION ON TEST DATA
# ======================================================
model.eval()
correct, total = 0, 0
with torch.no_grad():
    for images, labels in tqdm(test_loader, desc="Final Evaluation", unit="batch"):
        images, labels = images.to(device, non_blocking=True), labels.to(device, non_blocking=True)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

print(f"✅ Final Test Accuracy: {100 * correct / total:.2f}%")

# ======================================================
# PLOTTING
# ======================================================
plt.figure(figsize=(10,5))
plt.subplot(1,2,1)
plt.plot(train_losses, label="Train Loss")
plt.plot(val_losses, label="Val Loss")
plt.title("Loss per Epoch")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.grid(True)

plt.subplot(1,2,2)
plt.plot(train_accs, label="Train Acc")
plt.plot(val_accs, label="Val Acc")
plt.title("Accuracy per Epoch")
plt.xlabel("Epoch")
plt.ylabel("Accuracy (%)")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# ======================================================
# PREDICTION FUNCTION
# ======================================================
def predict_emotion(img_path):
    model.eval()
    emotions = np.load(emotions_path, allow_pickle=True).tolist()
    img = Image.open(img_path).convert("RGB")
    img_tensor = val_transform(img).unsqueeze(0).to(device)

    with torch.no_grad(), torch.cuda.amp.autocast():
        outputs = model(img_tensor)
        _, predicted = torch.max(outputs, 1)
    return emotions[predicted.item()]

# Example test
test_image = r"K:\Code\Project\Research Paper\Emotion Detection\test5.jpg"
print("Predicted Emotion:", predict_emotion(test_image))


✅ Loaded 35887 images | 7 emotion classes: ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']


C:\Users\Administrator\AppData\Local\Temp\ipykernel_20928\3635724056.py:123: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()  # Mixed precision training


🧠 Training model from scratch...


Epoch 1/30 [Training]:   0%|          | 0/477 [00:00<?, ?batch/s]C:\Users\Administrator\AppData\Local\Temp\ipykernel_20928\3635724056.py:144: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
Epoch 1/30 [Validation]:   0%|          | 0/85 [00:00<?, ?batch/s]C:\Users\Administrator\AppData\Local\Temp\ipykernel_20928\3635724056.py:171: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
Epoch 1/30 [Validation]: 100%|██████████| 85/85 [00:16<00:00,  5.23batch/s, acc=61.33%, loss=1.0225]



📊 Epoch 1/30 Summary:
   🏋️ Train -> Loss: 1.2504, Acc: 52.91%
   🔍 Val   -> Loss: 1.0225, Acc: 61.33%

💾 Best model saved (Val Acc: 61.33%)


...


📊 Epoch 9/30 Summary:
   🏋️ Train -> Loss: 0.0607, Acc: 98.05%
   🔍 Val   -> Loss: 1.7756, Acc: 60.83%



Epoch 10/30 [Validation]: 100%|██████████| 85/85 [00:16<00:00,  5.24batch/s, acc=61.29%, loss=1.8066]



📊 Epoch 10/30 Summary:
   🏋️ Train -> Loss: 0.0669, Acc: 97.85%
   🔍 Val   -> Loss: 1.8066, Acc: 61.29%



Epoch 11/30 [Training]:   0%|          | 0/477 [00:08<?, ?batch/s]


KeyboardInterrupt: 